In [76]:
using Oscar

In [77]:
# Creamos en anillo de polinomios
R, x = PolynomialRing(QQ, ["x$i" for i in 1:5])

(Multivariate Polynomial Ring in x1, x2, x3, x4, x5 over Rational Field, fmpq_mpoly[x1, x2, x3, x4, x5])

In [78]:
# Los polinomios a verificar
f1 = x[1]^2 - 1
f2 = x[1] + x[2]
f3 = x[1] + x[3]
f4 = x[2] + x[3]
listap = [f1,f2,f3,f4]

4-element Vector{fmpq_mpoly}:
 x1^2 - 1
 x1 + x2
 x1 + x3
 x2 + x3

In [79]:
listap =[x[1]^3 - 1
,x[2]^3 - 1
,x[3]^3 - 1
,x[1]^2 + x[1]*x[2] + x[2]^2
,x[1]^2 + x[1]*x[3] + x[3]^2]

9-element Vector{fmpq_mpoly}:
 x1^3 - 1
 x2^3 - 1
 x3^3 - 1
 x4^3 - 1
 x5^3 - 1
 x1^2 + x1*x2 + x2^2
 x1^2 + x1*x3 + x3^2
 x2^2 + x2*x4 + x4^2
 x2^2 + x2*x5 + x5^2

In [80]:
#  Calcula las particiones de d en sumas de n numeros, pueden ser 0
function partition(n::Int, d::Int)
    if n == 1
        return [[d]]
    end
    partitions = []
    for i in 0:d
        for p in partition(n-1, d-i)
            push!(partitions, [i; p])
        end
    end
    return partitions
end

partition (generic function with 1 method)

In [81]:
partition(3,3)

10-element Vector{Any}:
 [0, 0, 3]
 [0, 1, 2]
 [0, 2, 1]
 [0, 3, 0]
 [1, 0, 2]
 [1, 1, 1]
 [1, 2, 0]
 [2, 0, 1]
 [2, 1, 0]
 [3, 0, 0]

In [82]:
# Calcula los polinomios beta_i sin coeficientes
function polysBi(n::Int, d::Int)
    v = []
    for j in 0:d
        v = vcat(v, [reduce(*,[x[i]^par[i] for i in 1:length(par)]) for par in partition(n,j)])
    end
    return v
end

polysBi (generic function with 1 method)

In [83]:
polysBi(3,3)

20-element Vector{Any}:
 1
 x3
 x2
 x1
 x3^2
 x2*x3
 x2^2
 x1*x3
 x1*x2
 x1^2
 x3^3
 x2*x3^2
 x2^2*x3
 x2^3
 x1*x3^2
 x1*x2*x3
 x1*x2^2
 x1^2*x3
 x1^2*x2
 x1^3

In [84]:
# Retorna los polinomios beta_i * f_i sin loscoeficientes
function creaCertv(d,listap)
    v=[]
    n=length(gens(parent(listap[1])))
    for f in listap
        dd = total_degree(f)
        v = vcat(v,[ f * bi for bi in polysBi(n, d - dd)])
    end
    return v
end

creaCertv (generic function with 1 method)

In [85]:
d = maximum([total_degree(poly) for poly in listap])
bifi = creaCertv(d,listap)

29-element Vector{Any}:
 x1^3 - 1
 x2^3 - 1
 x3^3 - 1
 x4^3 - 1
 x5^3 - 1
 x1^2 + x1*x2 + x2^2
 x1^2*x5 + x1*x2*x5 + x2^2*x5
 x1^2*x4 + x1*x2*x4 + x2^2*x4
 x1^2*x3 + x1*x2*x3 + x2^2*x3
 x1^2*x2 + x1*x2^2 + x2^3
 ⋮
 x2^2*x3 + x2*x3*x4 + x3*x4^2
 x2^3 + x2^2*x4 + x2*x4^2
 x1*x2^2 + x1*x2*x4 + x1*x4^2
 x2^2 + x2*x5 + x5^2
 x2^2*x5 + x2*x5^2 + x5^3
 x2^2*x4 + x2*x4*x5 + x4*x5^2
 x2^2*x3 + x2*x3*x5 + x3*x5^2
 x2^3 + x2^2*x5 + x2*x5^2
 x1*x2^2 + x1*x2*x5 + x1*x5^2

In [87]:
function nulla(polys,K)
    n = length(gens(parent(polys[1])))
    d = maximum([total_degree(poly) for poly in polys])
    while d <= K
        # Creacion del polinomio de  setificado CERT
        CERTsinC = creaCertv(d,polys)
        nUnkows = length(CERTsinC)
        T, y =  PolynomialRing(R,["y$i" for i in 1:nUnkows])
        R2, z = PolynomialRing(T,["z$i" for i in 1:n])
        CERTsinC2 = [evaluate(bf,collect(gens(R2))) for bf in CERTsinC]
        CERTv = [ y[i]*CERTsinC2[i] for i in  1:length(CERTsinC2)]
        CERT = sum(CERTv)
        CERTt = collect(terms(CERT))

        # Creacion del sistema lineal 
        A = []
        for cee in CERTt
            achiquita = vec([Int( numerator(evaluate( coeff(cc, yi),zeros(Int, n)))) for yi in gens(T), cc in collect(coeffs(cee))])
            if isempty(A)
                A = achiquita
            else
                A=hcat(A,achiquita)
            end
        end
        A = transpose( A)

        b = zeros(Int,length(CERTt))
        b[end]+=1
        sol = A \ b
        if any(sol .!= 0) # si no es todo 0, el sistema es consistente
            println("The system of equations F is infeasible")
            return sol
        end
        println("Una iteracion")
        d += 1
    end
    println("The system of equations F is feasible.")
    return true
end

nulla (generic function with 1 method)

In [88]:
nulla(listap,10)

The system of equations F is infeasible


29-element Vector{Float64}:
 -0.18714643964210395
 -0.19809754314420824
 -0.16230087236223273
 -0.16800129053774024
 -0.1680012905377404
 -9.704665347072193e-18
 -0.015066050438516693
 -0.015066050438516668
 -0.017249536009316262
  0.038740031057638034
  ⋮
  0.00431238400232907
  0.021452474155297864
 -0.01834127879471594
  9.552386995867697e-18
  0.051548726761765924
 -0.014483787619636783
  0.0043123840023290725
  0.02145247415529805
 -0.01834127879471595